In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

In [2]:
# Firmographic file
df_firmographic = pd.read_csv("Clean Data/firmographic_clean_20230323.csv", dtype={'B2B_ADDR_ZIP5':'str'})

/var/folders/74/hvvyjn317tj9gl3n3gpqs3fw0000gn/T/ipykernel_22592/3729650555.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_firmographic = pd.read_csv("Clean Data/firmographic_clean_20230323.csv", dtype={'B2B_ADDR_ZIP5':'str'})


In [3]:
# Usage file
df_usage = pd.read_csv("Clean Data/usage_clean_20230420.csv")

In [4]:
#Join 2 files
df = df_firmographic.merge(df_usage, on='unique_identifier', how='right')

In [5]:
#Drop NA state
df = df.dropna(subset=['B2B_ADDR_STATE'])

In [6]:
#Drop low states
US_state = df['B2B_ADDR_STATE'].unique()
US_state = list(US_state)
remove_state = ['CA', 'DE', 'GA','IN','KY','ME','MI','RI','VA', 'DC', 'NH']
state = [i for i in US_state if i not in remove_state]

In [7]:
#Filtered dataframe
df_join= df.loc[df['B2B_ADDR_STATE'].isin(state)]

In [8]:
#Remove bankruptcy
df_bankrupt = df_join[['unique_identifier', 'B2B_BANKRUPTCY_FLAG']].copy()
df_bankrupt = df_bankrupt.dropna()
bankrupt_list = df_bankrupt['unique_identifier'].values.tolist()
df_no_bankrupt = df_join[~df_join['unique_identifier'].isin(bankrupt_list)]

In [9]:
#Define function to find missing values for each column
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    if str(mis_val_table_ren_columns.shape[0]) == '1':
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There is " + str(mis_val_table_ren_columns.shape[0]) +
            " column that have missing values.")
    else:
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [10]:
# Return the columns' names and the number of missing values
missing_values_table(df_no_bankrupt)

Your selected dataframe has 103 columns.
There are 75 columns that have missing values.


,Missing Values,% of Total Values
B2B_BANKRUPTCY_DATE,49439,100.0
B2B_BANKRUPTCY_FLAG,49439,100.0
B2B_FORTUNE_RANK,49436,100.0
B2B_FOREIGN_PARENT_FLAG,49432,100.0
B2B_IMPORT_EXPORT_FLAG,49231,99.6
...,...,...
B2B_CREDIT_YEAR,23,0.0
B2B_CREDIT_GRADE,12,0.0
B2B_NAICS_CODE,1,0.0
B2B_NAICS_CODE_DESCRIP,1,0.0


In [11]:
# Get the columns with > 80% missing
missing_df = missing_values_table(df_no_bankrupt);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 80].index)
if len(missing_columns) == 1: 
  print('We will remove %d column.' % len(missing_columns))
else:
  print('We will remove %d columns.' % len(missing_columns))
df_firmographic_dropped = df_firmographic.drop(list(missing_columns), axis = 1)

Your selected dataframe has 103 columns.
There are 75 columns that have missing values.
We will remove 35 columns.


In [12]:
#Export to csv
df_no_bankrupt.to_csv('Clean Data/ready_dataset_20230420.csv', index = False)